# Новый раздел

In [5]:
!pip install --upgrade sentence-transformers datasets transformers torch tensorboard

In [3]:
!pip uninstall -y torchvision


Found existing installation: torchvision 0.22.1+cu118
Uninstalling torchvision-0.22.1+cu118:
  Successfully uninstalled torchvision-0.22.1+cu118


In [4]:
!pip cache purge

Files removed: 2


In [4]:
!pip install --no-cache-dir \
  torchvision --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 68.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.


In [1]:
import torch

from sentence_transformers import SentenceTransformer, SentenceTransformerModelCardData, SentenceTransformerTrainingArguments, SentenceTransformerTrainer
from sentence_transformers.evaluation import InformationRetrievalEvaluator, SequentialEvaluator
from sentence_transformers.util import cos_sim
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers

from datasets import load_dataset, concatenate_datasets

In [ ]:
from huggingface_hub import login

# Запрос ввода токена (либо передать token="ваш_токен")
login()

In [3]:
!pip install --upgrade datasets


In [53]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/decoded_queries.csv")



In [54]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'references', 'corpus_id'],
        num_rows: 184
    })
})

In [55]:
# Clean & Format Columns
dataset = dataset.rename_column("question", "anchor")
dataset = dataset.rename_column("references", "positive")
dataset = dataset.remove_columns(["corpus_id"]) # keep global_chunk_id

In [56]:
dataset

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 184
    })
})

In [57]:
from datasets import DatasetDict

def add_id(example, idx):
    return {"id": idx}

dataset = dataset.map(
    add_id,
    with_indices=True,      # передаёт порядковый номер записи в idx
    remove_columns=[],      # не удаляем ничего автоматически
    desc="Adding id column"
)

Adding id column:   0%|          | 0/184 [00:00<?, ? examples/s]

In [58]:
dataset = dataset["train"]

In [59]:
dataset

Dataset({
    features: ['anchor', 'positive', 'id'],
    num_rows: 184
})

In [60]:
# Shuffle Dataset
dataset = dataset.shuffle()

# Split Dataset Into a 90/10 Train/Test split
dataset = dataset.train_test_split(test_size=0.1)

# Для каждого сплита ('train', 'test' и т.д.) добавляем колонку
for split in dataset.keys():
    # Берём список значений id из текущего сплита
    ids = dataset[split]["id"]

# Save Datasets to Disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

58515

In [61]:
dataset

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'id'],
        num_rows: 165
    })
    test: Dataset({
        features: ['anchor', 'positive', 'id'],
        num_rows: 19
    })
})

In [31]:
!pip install torchvision timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 68.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.


In [32]:
# Hugging Face model ID
model_id = "intfloat/multilingual-e5-small"

# Loading via SentenceTransformer
# model = SentenceTransformer(
#     model_id, model_kwargs={'torch_dtype': torch.float16}, device="cuda" if torch.cuda.is_available() else "cpu"
# )

model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Данный фрагмент кода загружает тренировочный и тестовый датасеты из JSON-файлов, объединяет их в единый корпус для оценки информационно-поисковых моделей, а затем формирует три ключевых структуры:
	1.	corpus — словарь, где ключи (id) соответствуют уникальным идентификаторам текстовых фрагментов, а значения — сами фрагменты;
	2.	queries — словарь, задающий соответствие между идентификаторами запросов и текстом вопросов;
	3.	relevant_docs — словарь, отображающий, какие документы в корпусе считаются релевантными для каждого запроса. Эти структуры используются в InformationRetrievalEvaluator для вычисления метрик качества поиска (MRR, Recall@k, NDCG и др.).

In [62]:
test_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [63]:
corpus_dataset = concatenate_datasets([train_dataset, test_dataset])

	•	Получаем списки идентификаторов id и соответствующих текстовых фрагментов positive.
	•	С помощью dict(zip(...)) создаём отображение id → текст для всего корпуса, что требуется InformationRetrievalEvaluator  ￼.

In [64]:
corpus = dict(
    zip(corpus_dataset["id"], corpus_dataset["positive"])
)

Аналогично строится словарь id запроса → текст вопроса, где берутся данные из test_dataset  ￼.

In [65]:
queries = dict(
    zip(test_dataset["id"], test_dataset["anchor"])
)

	•	relevant_docs — ключи q_id (идентификаторы запросов), значения — списки c id, где global_chunk_id совпадает, то есть документ считается релевантным для данного запроса.
	•	Такой формат (словарь из query_id → set(document_ids)) ожидает InformationRetrievalEvaluator для вычисления метрик качества ранжирования  ￼ ￼.

In [66]:
dataset

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'id'],
        num_rows: 165
    })
    test: Dataset({
        features: ['anchor', 'positive', 'id'],
        num_rows: 19
    })
})

In [67]:
# Create a mapping of relevant document (1 in our case) for each query
relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for q_id in queries:
    relevant_docs[q_id] = [q_id]


In [ ]:
# relevant_docs = {}
# for q_id, global_chunk_id in zip(test_dataset["id"], test_dataset["global_chunk_id"]):
#     if q_id not in relevant_docs:
#         relevant_docs[q_id] = []

#     matching_corpus_ids = [
#         cid for cid, chunk in zip(corpus_dataset["id"], corpus_dataset["global_chunk_id"])
#         if chunk == global_chunk_id
#     ]
#     relevant_docs[q_id].extend(matching_corpus_ids)

	•	Здесь задаётся список «уровней вложенности» эмбеддингов от полной размерности (обычно 768) до более компактных (64).
	•	Такая схема часто используется в Matryoshka-подходе, когда обучается и тестируется модель на нескольких вложенных по размеру встраиваниях.


In [68]:
matryoshka_dimensions = [384, 256, 128, 64]

	1.	Цикл по matryoshka_dimensions: для каждой указанной размерности создаётся свой оценщик.
	2.	InformationRetrievalEvaluator — стандартный класс из библиотеки Sentence-Transformers для вычисления IR-метрик (MRR, Recall@k, NDCG и др.) по парам «запрос – документ»  ￼.
	3.	truncate_dim=dim: этот параметр указывает усечь матрицу эмбеддингов до первых dim координат перед расчётом метрик  ￼.
	4.	score_functions={"cosine": cos_sim}: метод оценки «косинусное сходство» из утилит SBERT.

In [69]:
matryoshka_evaluators = []
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=corpus,
        relevant_docs=relevant_docs,
        name=f"dim_{dim}",
        truncate_dim=dim,
        score_functions={"cosine": cos_sim},
    )
    matryoshka_evaluators.append(ir_evaluator)

In [70]:
evaluator = SequentialEvaluator(matryoshka_evaluators)

In [71]:
# Evaluate the model
base_results = evaluator(model)

# Print header
print("\nBase Model Evaluation Results")
print("-" * 85)
print(f"{'Metric':15} {'384d':>12} {'256d':>12} {'128d':>12} {'64d':>12}")
print("-" * 85)

# List of metrics to display
metrics = [
    'ndcg@10',
    'mrr@10',
    'map@100',
    'accuracy@1',
    'accuracy@3',
    'accuracy@5',
    'accuracy@10',
    'precision@1',
    'precision@3',
    'precision@5',
    'precision@10',
    'recall@1',
    'recall@3',
    'recall@5',
    'recall@10'
]

# Print each metric
for metric in metrics:
    values = []
    for dim in matryoshka_dimensions:
        key = f"dim_{dim}_cosine_{metric}"
        values.append(base_results[key])

    # Highlight NDCG@10
    metric_name = f"=={metric}==" if metric == "ndcg@10" else metric
    print(f"{metric_name:15}", end="  ")
    for val in values:
        print(f"{val:12.4f}", end=" ")
    print()

# Print sequential score
print("-" * 85)
print(f"{'seq_score:'} {base_results['sequential_score']:1f}")


Base Model Evaluation Results
-------------------------------------------------------------------------------------
Metric                  384d         256d         128d          64d
-------------------------------------------------------------------------------------
==ndcg@10==            0.6952       0.6523       0.5706       0.5283 
mrr@10                 0.6645       0.6228       0.5149       0.4789 
map@100                0.6746       0.6321       0.5218       0.4837 
accuracy@1             0.5789       0.5263       0.3684       0.3684 
accuracy@3             0.7368       0.7368       0.6316       0.5789 
accuracy@5             0.7368       0.7368       0.7368       0.5789 
accuracy@10            0.7895       0.7368       0.7368       0.6842 
precision@1            0.5789       0.5263       0.3684       0.3684 
precision@3            0.2456       0.2456       0.2105       0.1930 
precision@5            0.1474       0.1474       0.1474       0.1158 
precision@10           0.0789

In [72]:
# load model with SDPA for using Flash Attention 2
model = SentenceTransformer(
    model_id,
    model_kwargs={"attn_implementation": "sdpa"},
    model_card_data=SentenceTransformerModelCardData(
        language="rus",
        license="apache-2.0",
        model_name="multilingual-e5-small Embed base Legal Matryoshka",
    ),
)

In [73]:
# Initial Loss
base_loss = MultipleNegativesRankingLoss(model)

# Matryoshka Loss Wrapper
train_loss = MatryoshkaLoss(
    model, base_loss, matryoshka_dims=matryoshka_dimensions
)

In [74]:
# Training Arguments
args = SentenceTransformerTrainingArguments(
    output_dir="intfloat/multilingual-e5-small-legal-matryoshka", # output directory and hugging face model ID
    num_train_epochs=8,                                           # number of epochs
    per_device_train_batch_size=16,                               # train batch size
    gradient_accumulation_steps=8,                               # for a global batch size of 512
    per_device_eval_batch_size=8,                                # evaluation batch size
    warmup_ratio=0.1,                                             # warmup ratio
    learning_rate=2e-5,                                           # learning rate, 2e-5 is a good value
    lr_scheduler_type="cosine",                                   # use cosine learning rate scheduler
    optim="adamw_torch_fused",                                    # use fused adamw optimizer
    tf32=False,                                                    # use tf32 precision
    bf16=True,                                                    # use bf16 precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,                    # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="epoch",                                        # evaluate after each epoch
    save_strategy="epoch",                                        # save after each epoch
    logging_steps=10,                                             # log every 10 steps
    save_total_limit=3,                                           # save only the last 3 models
    load_best_model_at_end=True,                                  # load the best model when training ends
    metric_for_best_model="eval_dim_128_cosine_ndcg@10",          # Optimizing for the best ndcg@10 score for the 128 dimension
    report_to="none"                                              # Turning off training logging for now, input 'wandb' etc. if desired.
)

In [75]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset.select_columns(["anchor","positive"]),
    loss=train_loss,
    evaluator=evaluator,
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [76]:
import torch
torch.cuda.empty_cache()

In [77]:
# Start training
trainer.train()

# Save the best model based on our eval_dim_128_cosine_ndcg@10 criteria
trainer.save_model()

Epoch,Training Loss,Validation Loss,Dim 384 Cosine Accuracy@1,Dim 384 Cosine Accuracy@3,Dim 384 Cosine Accuracy@5,Dim 384 Cosine Accuracy@10,Dim 384 Cosine Precision@1,Dim 384 Cosine Precision@3,Dim 384 Cosine Precision@5,Dim 384 Cosine Precision@10,Dim 384 Cosine Recall@1,Dim 384 Cosine Recall@3,Dim 384 Cosine Recall@5,Dim 384 Cosine Recall@10,Dim 384 Cosine Ndcg@10,Dim 384 Cosine Mrr@10,Dim 384 Cosine Map@100,Dim 256 Cosine Accuracy@1,Dim 256 Cosine Accuracy@3,Dim 256 Cosine Accuracy@5,Dim 256 Cosine Accuracy@10,Dim 256 Cosine Precision@1,Dim 256 Cosine Precision@3,Dim 256 Cosine Precision@5,Dim 256 Cosine Precision@10,Dim 256 Cosine Recall@1,Dim 256 Cosine Recall@3,Dim 256 Cosine Recall@5,Dim 256 Cosine Recall@10,Dim 256 Cosine Ndcg@10,Dim 256 Cosine Mrr@10,Dim 256 Cosine Map@100,Dim 128 Cosine Accuracy@1,Dim 128 Cosine Accuracy@3,Dim 128 Cosine Accuracy@5,Dim 128 Cosine Accuracy@10,Dim 128 Cosine Precision@1,Dim 128 Cosine Precision@3,Dim 128 Cosine Precision@5,Dim 128 Cosine Precision@10,Dim 128 Cosine Recall@1,Dim 128 Cosine Recall@3,Dim 128 Cosine Recall@5,Dim 128 Cosine Recall@10,Dim 128 Cosine Ndcg@10,Dim 128 Cosine Mrr@10,Dim 128 Cosine Map@100,Dim 64 Cosine Accuracy@1,Dim 64 Cosine Accuracy@3,Dim 64 Cosine Accuracy@5,Dim 64 Cosine Accuracy@10,Dim 64 Cosine Precision@1,Dim 64 Cosine Precision@3,Dim 64 Cosine Precision@5,Dim 64 Cosine Precision@10,Dim 64 Cosine Recall@1,Dim 64 Cosine Recall@3,Dim 64 Cosine Recall@5,Dim 64 Cosine Recall@10,Dim 64 Cosine Ndcg@10,Dim 64 Cosine Mrr@10,Dim 64 Cosine Map@100,Sequential Score
1,No log,No log,0.578947,0.736842,0.736842,0.789474,0.578947,0.245614,0.147368,0.078947,0.578947,0.736842,0.736842,0.789474,0.696112,0.665414,0.675444,0.473684,0.736842,0.736842,0.736842,0.473684,0.245614,0.147368,0.073684,0.473684,0.736842,0.736842,0.736842,0.632827,0.596491,0.605936,0.421053,0.736842,0.736842,0.736842,0.421053,0.245614,0.147368,0.073684,0.421053,0.736842,0.736842,0.736842,0.613403,0.570175,0.577525,0.368421,0.631579,0.631579,0.684211,0.368421,0.210526,0.126316,0.068421,0.368421,0.631579,0.631579,0.684211,0.543408,0.497076,0.502277,0.543408
2,No log,No log,0.526316,0.736842,0.789474,0.789474,0.526316,0.245614,0.157895,0.078947,0.526316,0.736842,0.789474,0.789474,0.672613,0.633333,0.642957,0.473684,0.736842,0.736842,0.736842,0.473684,0.245614,0.147368,0.073684,0.473684,0.736842,0.736842,0.736842,0.639718,0.605263,0.614208,0.421053,0.684211,0.736842,0.736842,0.421053,0.228070,0.147368,0.073684,0.421053,0.684211,0.736842,0.736842,0.593665,0.545614,0.554822,0.368421,0.631579,0.631579,0.631579,0.368421,0.210526,0.126316,0.063158,0.368421,0.631579,0.631579,0.631579,0.534455,0.500000,0.509940,0.534455
3,No log,No log,0.526316,0.736842,0.736842,0.842105,0.526316,0.245614,0.147368,0.084211,0.526316,0.736842,0.736842,0.842105,0.693105,0.645614,0.650769,0.526316,0.736842,0.736842,0.736842,0.526316,0.245614,0.147368,0.073684,0.526316,0.736842,0.736842,0.736842,0.659143,0.631579,0.640681,0.421053,0.684211,0.736842,0.789474,0.421053,0.228070,0.147368,0.078947,0.421053,0.684211,0.736842,0.789474,0.611816,0.554094,0.559442,0.473684,0.578947,0.631579,0.631579,0.473684,0.192982,0.126316,0.063158,0.473684,0.578947,0.631579,0.631579,0.560459,0.536842,0.549640,0.560459
4,No log,No log,0.526316,0.736842,0.736842,0.842105,0.526316,0.245614,0.147368,0.084211,0.526316,0.736842,0.736842,0.842105,0.693735,0.646199,0.651342,0.578947,0.736842,0.736842,0.736842,0.578947,0.245614,0.147368,0.073684,0.578947,0.736842,0.736842,0.736842,0.678568,0.657895,0.668354,0.421053,0.684211,0.736842,0.789474,0.421053,0.228070,0.147368,0.078947,0.421053,0.684211,0.736842,0.789474,0.612413,0.554386,0.560137,0.473684,0.578947,0.631579,0.684211,0.473684,0.192982,0.126316,0.068421,0.473684,0.578947,0.631579,0.684211,0.575672,0.542105,0.553108,0.575672
5,29.141400,No log,0.526316,0.789474,0.789474,0.842105,0.526316,0.263158,0.157895,0.084211,0.526316,0.789474,0.789474,0.842105,0.701303,0.654971,0.660712,0.578947,0.736842,0.736842,0.736

In [79]:
# push model to hub
trainer.model.push_to_hub("multilingual-e5-small-legal-matryoshka")


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

'https://huggingface.co/SergeyKarpenko1/multilingual-e5-small-legal-matryoshka/commit/14e78fc04bdf22b3d28d1b5a7da4dbf5adf26d35'

In [96]:
output_dir='/content/intfloat/multilingual-e5-small-legal-matryoshka'

In [97]:
fine_tuned_model = SentenceTransformer(
    args.output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)

# Evaluate the model
ft_results = evaluator(fine_tuned_model)

# Print header
print("Fine Tuned Model Evaluation Results")
print("-" * 85)
print(f"{'Metric':15} {'384d':>12} {'256d':>12} {'128d':>12} {'64d':>12}")
print("-" * 85)

# List of metrics to display
metrics = [
    'ndcg@10',
    'mrr@10',
    'map@100',
    'accuracy@1',
    'accuracy@3',
    'accuracy@5',
    'accuracy@10',
    'precision@1',
    'precision@3',
    'precision@5',
    'precision@10',
    'recall@1',
    'recall@3',
    'recall@5',
    'recall@10'
]

# Print each metric
for metric in metrics:
    values = []
    for dim in matryoshka_dimensions:
        key = f"dim_{dim}_cosine_{metric}"
        values.append(ft_results[key])

    # Highlight NDCG@10
    metric_name = f"=={metric}==" if metric == "ndcg@10" else metric
    print(f"{metric_name:15}", end="  ")
    for val in values:
        print(f"{val:12.4f}", end=" ")
    print()

# Print sequential score
print("-" * 85)
print(f"{'seq_score:'} {ft_results['sequential_score']:1f}")

Fine Tuned Model Evaluation Results
-------------------------------------------------------------------------------------
Metric                  384d         256d         128d          64d
-------------------------------------------------------------------------------------
==ndcg@10==            0.7013       0.6980       0.6302       0.5776 
mrr@10                 0.6550       0.6842       0.5789       0.5439 
map@100                0.6607       0.6968       0.5850       0.5573 
accuracy@1             0.5263       0.6316       0.4737       0.4737 
accuracy@3             0.7895       0.7368       0.6842       0.5789 
accuracy@5             0.7895       0.7368       0.6842       0.5789 
accuracy@10            0.8421       0.7368       0.7895       0.6842 
precision@1            0.5263       0.6316       0.4737       0.4737 
precision@3            0.2632       0.2456       0.2281       0.1930 
precision@5            0.1579       0.1474       0.1368       0.1158 
precision@10           0

In [98]:
from prettytable import PrettyTable

# Предположим, что у вас есть два словаря результатов:
# base_results и ft_results — они имеют одинаковые ключи, вида "dim_{d}_cosine_{metric}"
# и содержат значения метрик.

# Задаём интересующие нас размерности и метрики
matryoshka_dimensions = [384, 256, 128, 64]
metrics = ['ndcg@10', 'mrr@10', 'map@100']

# Инициализируем таблицу
table = PrettyTable()
table.field_names = [
    "Metric", "Dimension", "Base", "Fine-tuned", "Abs. Improvement", "% Improvement"
]

for metric in metrics:
    for dim in matryoshka_dimensions:
        key = f"dim_{dim}_cosine_{metric}"
        base_val = base_results.get(key, 0.0)
        ft_val   = ft_results.get(key,   0.0)
        abs_imp  = ft_val - base_val
        pct_imp  = (abs_imp / base_val * 100) if base_val else 0.0

        # Форматируем проценты с одним знаком после запятой
        pct_str = f"{pct_imp:.1f}%"

        table.add_row([
            metric,
            f"{dim}d",
            f"{base_val:.4f}",
            f"{ft_val:.4f}",
            f"{abs_imp:.4f}",
            pct_str
        ])

print("Base vs FT Comparison")
print(table)

Base vs FT Comparison
+---------+-----------+--------+------------+------------------+---------------+
|  Metric | Dimension |  Base  | Fine-tuned | Abs. Improvement | % Improvement |
+---------+-----------+--------+------------+------------------+---------------+
| ndcg@10 |    384d   | 0.6952 |   0.7013   |      0.0061      |      0.9%     |
| ndcg@10 |    256d   | 0.6523 |   0.6980   |      0.0457      |      7.0%     |
| ndcg@10 |    128d   | 0.5706 |   0.6302   |      0.0596      |     10.5%     |
| ndcg@10 |    64d    | 0.5283 |   0.5776   |      0.0493      |      9.3%     |
|  mrr@10 |    384d   | 0.6645 |   0.6550   |     -0.0095      |     -1.4%     |
|  mrr@10 |    256d   | 0.6228 |   0.6842   |      0.0614      |      9.9%     |
|  mrr@10 |    128d   | 0.5149 |   0.5789   |      0.0640      |     12.4%     |
|  mrr@10 |    64d    | 0.4789 |   0.5439   |      0.0649      |     13.6%     |
| map@100 |    384d   | 0.6746 |   0.6607   |     -0.0139      |     -2.1%     |
| map@

In [122]:
model_1 = model

In [123]:
model_2 = fine_tuned_model

In [124]:
sentences = [
    'Какие пункты Гражданского кодекса РФ были применены судами при удовлетворении исковых требований в уменьшенном размере?',
    '[{"content": "начислением процентов с 02.10.2024 по день фактической оплаты.\\nУдовлетворяя исковые требования ООО \\"ЗОРКИЙ ВЗГЛЯД\\" в уменьшенном размере, руководствуясь положениямистатей 309,310,395,702-729,779-781 Гражданского кодекса Российской Федерации, суды первой и апелляционной инстанций, установив факт оказания услуг, признали исковые требования о взыскании задолженности обоснованными по праву, ", "start_index": 259926, "end_index": 260316}]',
    '[{"content": "можно получить на официальном Интернет-сайте Арбитражного суда Московского округа www.fasmo.arbitr.ru или по телефону справочной службы суда (495) 609-57-75.\\nСудья В.В. ", "start_index": 192735, "end_index": 192904}]',
]

In [125]:
embeddings = model_1.encode(sentences)
print(embeddings.shape)


(3, 384)


In [126]:
# Get the similarity scores for the embeddings
similarities = model_1.similarity(embeddings, embeddings)
print(similarities.shape)
# [3, 3]

torch.Size([3, 3])


In [127]:
# Get the similarity scores for the embeddings
similarities = model_1.similarity(embeddings, embeddings)
print(similarities[0])

tensor([1.0000, 0.7497, 0.6505])


In [128]:
embeddings = model_2.encode(sentences)
print(embeddings.shape)

(3, 384)


In [129]:
# Get the similarity scores for the embeddings
similarities = model_2.similarity(embeddings, embeddings)
print(similarities[0])

tensor([1.0000, 0.7508, 0.6512])


In [130]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model_246d = SentenceTransformer("SergeyKarpenko1/multilingual-e5-small-legal-matryoshka", truncate_dim=256)

In [131]:
embeddings = model_246d.encode(sentences)
print(embeddings.shape)

(3, 256)


In [132]:
# Get the similarity scores for the embeddings
similarities = model_64d.similarity(embeddings, embeddings)
print(similarities[0])

tensor([1.0000, 0.7402, 0.6130])


Мы видим что дообучение даже небольшой модели всего на 184 парах вопрос/ответ повышает производительность от 1 до 15% в зависимости от размерности! За файнтюненая модель размерности 256 дает даже лучший результат чем базовая 384 размерности.
Это позволяет сократить затраты на хранение и улучшить поиск по косинусному сходству.
